<a href="https://colab.research.google.com/github/jjgp/colab/blob/main/notebooks/ngrok.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# TODO: Fill-in credentials below

AUTHORIZED_KEY = """
ssh-ed25519 XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX+XXXXXXXXXXXXX XXXXXXX@XXXXX.XXX
""".strip()  # noqa
AUTHTOKEN = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"

# SSH

In [ ]:
%%bash -e
apt-get install openssh-server > /dev/null
sed -i -E "s/#?PermitUserEnvironment.*/PermitUserEnvironment yes/" /etc/ssh/sshd_config
mkdir -p ~/.ssh

In [ ]:
with open("/root/.ssh/authorized_keys", "w") as f:
    f.write(AUTHORIZED_KEY)

Exporting the `env` with logic taken from [microsoft/node-pty](https://github.com/microsoft/node-pty):
- [src/unixTerminal.ts#L282](https://github.com/microsoft/node-pty/blob/1674722e1caf3ff4dd52438b70ed68d46af83a6d/src/unixTerminal.ts#L282)
- [src/terminal.ts#L197](https://github.com/microsoft/node-pty/blob/1674722e1caf3ff4dd52438b70ed68d46af83a6d/src/terminal.ts#L197)

In [ ]:
import os

# Some of these are likely not to happen based on how the process is started...
envvars_to_skip = {
    "TMUX",
    "TMUX_PANE",
    "STY",
    "WINDOW",
    "WINDOWID",
    "TERMCAP",
    "COLUMNS",
    "LINES",
}

with open("/root/.ssh/environment", "w") as f:
    for envvar, value in os.environ.items():
        if envvar not in envvars_to_skip:
            f.write(f"{envvar}={value}\n")

In [ ]:
!service ssh restart > /dev/null

# ngrok

In [ ]:
%%bash -e
curl -sSLO https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
tar -xzf ngrok-stable-linux-amd64.tgz
mv ngrok /usr/local/bin
rm ngrok-stable-linux-amd64.tgz

In [ ]:
!ngrok authtoken $AUTHTOKEN > /dev/null

In [ ]:
%%bash --bg
ngrok tcp 22 --log ngrok.log --log-format logfmt

Wait a couple seconds and run the following to get the url...

In [ ]:
!tac ngrok.log | grep -m1 url=tcp://

In [ ]:
!killall -SIGQUIT -q ngrok || true

To connect to the instance...

```sh
ssh -i [KEYFILE PATH] root@[NGROK DOMAIN] -p [NGROK PORT]
```

...or with the `~/.ssh/config`...

```
Host [NGROK DOMAIN]
  HostName [NGROK DOMAIN]
  IdentityFile [KEYFILE PATH]
  Port [NGROK PORT] 
  User root
```

...and simply run:

```sh
ssh root@[NGROK DOMAIN]
```